# Ant Colony Optimization (ACO) for Traveling Salesman Problem (TSP)

This Colab notebook demonstrates the Ant Colony Optimization (ACO) algorithm applied to a Traveling Salesman Problem (TSP). It includes functionalities for defining a graph, running the ACO algorithm, and visualizing the results, including pheromone trails and the best-found tour.

A key feature showcased in the second part of the script is the ACO algorithm's ability to efficiently re-compute an optimal solution when an edge in the graph is removed, leveraging the existing pheromone matrix.

## 1. Setup and Imports

First, we'll import the necessary libraries. We are strictly using `numpy` for numerical operations and `matplotlib` for plotting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Although imported, this is not directly used in the provided Matlab code's plotting.
import math
import time

# Set a random seed for reproducibility
np.random.seed(42)

# --- Define Helper Functions ---

def average(a, b):
    """
    Calculates the average of two numbers.

    Parameters:
    a (float): The first number.
    b (float): The second number.

    Returns:
    float: The average of a and b.
    """
    return (a + b) / 2.0

def roulette_wheel(probabilities):
    """
    Selects an index based on a roulette wheel selection mechanism.
    The probability array determines the likelihood of selecting each index.
    Must sum to 1.

    Parameters:
    probabilities (np.ndarray): A 1D NumPy array of probabilities.

    Returns:
    int: The selected index (0-based).
    """
    # Calculate the cumulative sum of probabilities
    cumulative_sum = np.cumsum(probabilities)
    # Generate a random number between 0 and 1
    r = np.random.rand()
    # Find the first index where the cumulative sum is greater than or equal to r
    # This simulates spinning a roulette wheel where larger probabilities
    # correspond to larger sections of the wheel.
    next_node_index = np.where(r <= cumulative_sum)[0][0]
    return next_node_index

# --- Graph and Edge Management ---

def add_edge(graph, i, j, weight):
    """
    Defines the weight on the edge connecting node i with node j.
    The graph is assumed to be undirected, so weights are set symmetrically.

    Parameters:
    graph (dict): The graph dictionary containing 'edges'.
    i (int): Index of the first node (0-based).
    j (int): Index of the second node (0-based).
    weight (float): The weight of the edge.
    """
    graph['edges'][i, j] = weight
    graph['edges'][j, i] = weight

def create_graph(x_coords, y_coords):
    """
    Creates the graph structure for the TSP problem.

    Parameters:
    x_coords (list): List of x-coordinates for each node.
    y_coords (list): List of y-coordinates for each node.

    Returns:
    dict: A dictionary representing the graph with 'n_nodes', 'nodes' (coordinates),
          and 'edges' (adjacency matrix).
    """
    n_nodes = len(x_coords)
    graph = {
        'n_nodes': n_nodes,
        'nodes': [{'x': x_coords[k], 'y': y_coords[k]} for k in range(n_nodes)],
        # Initialize all edge weights to infinity, representing no direct connection initially
        'edges': np.full((n_nodes, n_nodes), np.inf)
    }

    # Define specific edges and their weights as per the Matlab code
    # Note: Node indices are converted from 1-based (Matlab) to 0-based (Python)
    add_edge(graph, 0, 1, 8)
    add_edge(graph, 0, 2, 3)
    add_edge(graph, 0, 3, 5)
    add_edge(graph, 1, 2, 6)
    add_edge(graph, 1, 4, 1)
    add_edge(graph, 1, 5, 6)
    add_edge(graph, 2, 3, 4)
    add_edge(graph, 2, 5, 6)
    add_edge(graph, 2, 6, 5)
    add_edge(graph, 3, 6, 4)
    add_edge(graph, 3, 7, 5)
    add_edge(graph, 4, 5, 1)
    add_edge(graph, 4, 8, 9)
    add_edge(graph, 4, 9, 4)
    add_edge(graph, 5, 6, 7)
    add_edge(graph, 5, 9, 2)
    add_edge(graph, 5, 10, 9)
    add_edge(graph, 6, 7, 1)
    add_edge(graph, 6, 10, 10)
    add_edge(graph, 6, 11, 4)
    add_edge(graph, 7, 11, 5)
    add_edge(graph, 7, 12, 10)
    add_edge(graph, 8, 9, 8)
    add_edge(graph, 8, 13, 7)
    add_edge(graph, 9, 10, 5)
    add_edge(graph, 9, 13, 7)
    add_edge(graph, 9, 14, 2)
    add_edge(graph, 10, 11, 9)
    add_edge(graph, 10, 14, 8)
    add_edge(graph, 10, 15, 7)
    add_edge(graph, 11, 12, 3)
    add_edge(graph, 11, 15, 8)
    add_edge(graph, 11, 16, 5)
    add_edge(graph, 12, 16, 9)
    add_edge(graph, 13, 14, 6)
    add_edge(graph, 13, 17, 2)
    add_edge(graph, 14, 15, 8)
    add_edge(graph, 14, 17, 7)
    add_edge(graph, 14, 18, 9)
    add_edge(graph, 15, 16, 2)
    add_edge(graph, 15, 18, 9)
    add_edge(graph, 15, 19, 3)
    add_edge(graph, 16, 19, 5)
    add_edge(graph, 17, 18, 10)
    add_edge(graph, 17, 20, 4)
    add_edge(graph, 18, 19, 9)
    add_edge(graph, 18, 20, 3)
    add_edge(graph, 19, 20, 8)

    return graph

# Helper for midpoint calculations from the original Matlab code.
# This function's logic directly reflects the specific hardcoded edge definitions.
def get_midpoint_coordinates(x_coords, y_coords):
    """
    Calculates the midpoint coordinates for each predefined edge in the graph.
    This function explicitly lists the pairs of nodes that form the 48 edges
    and calculates their midpoints.

    Parameters:
    x_coords (list): X-coordinates of all nodes.
    y_coords (list): Y-coordinates of all nodes.

    Returns:
    tuple: Two lists, x_midpoints and y_midpoints, for each of the 48 defined edges.
    """
    # Define the 48 edges by their 0-based node indices
    edges_node_pairs = [
        (0, 1), (0, 2), (0, 3), (1, 2), (1, 4), (1, 5), (2, 3), (2, 5), (2, 6),
        (3, 6), (3, 7), (4, 5), (4, 8), (4, 9), (5, 6), (5, 9), (5, 10), (6, 7),
        (6, 10), (6, 11), (7, 11), (7, 12), (8, 9), (8, 13), (9, 10), (9, 13),
        (9, 14), (10, 11), (10, 14), (10, 15), (11, 12), (11, 15), (11, 16),
        (12, 16), (13, 14), (13, 17), (14, 15), (14, 17), (14, 18), (15, 16),
        (15, 18), (15, 19), (16, 19), (17, 18), (17, 20), (18, 19), (18, 20),
        (19, 20)
    ]

    x_midpoints = []
    y_midpoints = []

    for i, j in edges_node_pairs:
        x_midpoints.append(average(x_coords[i], x_coords[j]))
        y_midpoints.append(average(y_coords[i], y_coords[j]))

    return x_midpoints, y_midpoints

def find_closest_edge(click_x, click_y, x_midpoints_edges, y_midpoints_edges):
    """
    Identifies the edge whose midpoint is closest to the given (click_x, click_y) coordinates.

    Parameters:
    click_x (float): X-coordinate of the click.
    click_y (float): Y-coordinate of the click.
    x_midpoints_edges (list): List of x-coordinates of midpoints for all edges.
    y_midpoints_edges (list): List of y-coordinates of midpoints for all edges.

    Returns:
    int: The 0-based index of the closest edge in the `edges_node_pairs` list
         defined in `get_midpoint_coordinates`.
    """
    min_distance = np.inf
    deleted_edge_idx = -1

    # Iterate through all midpoint coordinates to find the closest one
    for i in range(len(x_midpoints_edges)):
        distance = np.sqrt((click_x - x_midpoints_edges[i])**2 + (click_y - y_midpoints_edges[i])**2)
        if distance < min_distance:
            min_distance = distance
            deleted_edge_idx = i
    return deleted_edge_idx

def get_nodes_from_edge_index(deleted_edge_idx):
    """
    Translates a predefined edge index (0-based) back to the two nodes it connects.
    This function serves as a lookup table, mirroring the logic of the original Matlab `findNode` function.

    Parameters:
    deleted_edge_idx (int): The 0-based index of the deleted edge.
    Returns:
    list: A list containing the two 0-based node indices that form the edge.
    """
    # This list must match the order of `edges_node_pairs` in `get_midpoint_coordinates`
    edges_node_pairs = [
        (0, 1), (0, 2), (0, 3), (1, 2), (1, 4), (1, 5), (2, 3), (2, 5), (2, 6),
        (3, 6), (3, 7), (4, 5), (4, 8), (4, 9), (5, 6), (5, 9), (5, 10), (6, 7),
        (6, 10), (6, 11), (7, 11), (7, 12), (8, 9), (8, 13), (9, 10), (9, 13),
        (9, 14), (10, 11), (10, 14), (10, 15), (11, 12), (11, 15), (11, 16),
        (12, 16), (13, 14), (13, 17), (14, 15), (14, 17), (14, 18), (15, 16),
        (15, 18), (15, 19), (16, 19), (17, 18), (17, 20), (18, 19), (18, 20),
        (19, 20)
    ]
    return list(edges_node_pairs[deleted_edge_idx])


# --- ACO Core Functions ---

def fitness_function(tour, graph):
    """
    Calculates the fitness (total length) of a given tour.
    The fitness is the sum of weights of edges in the tour.

    Parameters:
    tour (list): A list of node indices (0-based) representing the tour path.
    graph (dict): The graph dictionary containing 'edges'.

    Returns:
    float: The total length (fitness) of the tour. Returns infinity if any edge in the tour is infinite.
    """
    fitness = 0.0
    # Iterate through the tour to sum the weights of consecutive edges
    for i in range(len(tour) - 1):
        current_node = tour[i]
        next_node = tour[i+1]
        edge_weight = graph['edges'][current_node, next_node]
        if np.isinf(edge_weight):
            # If any edge in the tour does not exist (infinite weight), the tour is invalid.
            return np.inf
        fitness += edge_weight
    return fitness

def create_colony(graph, ant_no, tau, eta, alpha, beta):
    """
    Generates paths for each ant in the colony. Each ant constructs a complete tour
    based on pheromone levels (tau) and heuristic information (eta).

    Parameters:
    graph (dict): The graph structure.
    ant_no (int): Number of ants in the colony.
    tau (np.ndarray): Pheromone matrix.
    eta (np.ndarray): Heuristic information matrix (inverse of edge weights).
    alpha (float): Exponent for pheromone influence.
    beta (float): Exponent for heuristic influence.

    Returns:
    dict: A dictionary representing the colony, containing a list of ants and a 'queen' ant.
    """
    node_no = graph['n_nodes']
    colony = {'ants': [], 'queen': {'tour': [], 'fitness': np.inf}}

    for i in range(ant_no):
        ant = {'tour': [], 'fitness': np.inf}
        
        # Select a random starting node for the current ant
        initial_node = np.random.randint(0, node_no)
        ant['tour'].append(initial_node)
        current_node = initial_node
        
        # Keep track of visited nodes to avoid cycles in the first part of tour construction
        visited_nodes = {initial_node} 
        
        # Construct the tour until all nodes have been visited
        while len(visited_nodes) < node_no:
            current_node = ant['tour'][-1] # Get the current node (last node added to the tour)

            # Calculate probabilities for moving to the next node
            # Probability is proportional to (pheromone^alpha) * (heuristic^beta)
            probabilities_all_nodes = (tau[current_node, :] ** alpha) * (eta[current_node, :] ** beta)

            # Assign a very small probability to already visited nodes to discourage revisiting
            for visited_node in visited_nodes:
                probabilities_all_nodes[visited_node] = 1e-10 # Small epsilon to avoid division by zero if all others are 0

            # Set probability to zero for non-existent edges (infinite weight)
            non_existent_edges = np.isinf(graph['edges'][current_node, :])
            probabilities_all_nodes[non_existent_edges] = 0.0

            # Normalize probabilities to sum to 1
            sum_probabilities = np.sum(probabilities_all_nodes)
            if sum_probabilities == 0:
                # If no valid moves, break (this can happen if an ant gets trapped)
                break 
            
            probabilities = probabilities_all_nodes / sum_probabilities
            
            # Select the next node using roulette wheel selection
            next_node = roulette_wheel(probabilities)
            
            ant['tour'].append(next_node)
            visited_nodes.add(next_node) # Mark the node as visited

        # After visiting all nodes, complete the tour by returning to the initial node
        # The original Matlab code logic for completing the tour was slightly different (more complex iteration).
        # We simplify it by ensuring the last node connects back to the first.
        # However, the Matlab code also allows for visiting nodes multiple times in the closing path.
        # Replicating the "complete tour" logic from Matlab:
        # It seems the Matlab code was trying to make the ant return to the *initial_node*
        # while *potentially* traversing through other nodes if the current_node != initial_node
        # this is unusual for a strict TSP, but we will replicate the logic.
        
        # The Matlab code's 'completeTour' variable which was set to [] and then
        # `P_allNodes(completeTour) = 1e-7;` suggests it was trying to prevent
        # revisiting nodes specifically in the "return to start" phase, but `completeTour` was reset.
        # The simpler interpretation is to simply connect the last visited node to the initial node.
        # Let's re-read the Matlab createColony's second while loop carefully:
        # `while currentNode ~= initial_node` implies the ant keeps adding nodes until it reaches initial.
        # `P_allNodes(completeTour) = 1e-7;` inside this loop, but `completeTour` is always reset.
        # This implies that the ant attempts to go back to the initial node using the same pheromone/heuristic rule,
        # but without strict "no revisiting" constraint for already "completed" tours.
        # Given the context of TSP, the standard is to just close the loop.
        # However, to maintain fidelity to the original Matlab, if the ant's tour
        # isn't complete (doesn't include all nodes or doesn't end at start node),
        # we try to close it as the Matlab code does.

        # Ensure the tour connects back to the starting node to form a complete cycle (TSP requirement)
        if len(ant['tour']) == node_no and ant['tour'][-1] != initial_node:
            ant['tour'].append(initial_node)
        elif len(ant['tour']) < node_no:
            # This means the ant got stuck and couldn't visit all nodes. This tour is invalid.
            ant['tour'] = [] # Clear invalid tour
            ant['fitness'] = np.inf


        colony['ants'].append(ant) # Add the completed ant to the colony

    return colony


def update_pheromone(tau, colony):
    """
    Updates the pheromone matrix based on the paths taken by ants.
    Pheromone is deposited inversely proportional to the tour's fitness (length).

    Parameters:
    tau (np.ndarray): The current pheromone matrix.
    colony (dict): The colony dictionary containing ants and their tours/fitness.

    Returns:
    np.ndarray: The updated pheromone matrix.
    """
    # Iterate through each ant in the colony
    for ant in colony['ants']:
        # Only update pheromone for valid tours (finite fitness)
        if np.isinf(ant['fitness']):
            continue

        # Get the inverse of fitness (more pheromone for shorter tours)
        delta_tau = 1.0 / ant['fitness']

        # Iterate through the edges in the ant's tour and deposit pheromone
        for j in range(len(ant['tour']) - 1):
            current_node = ant['tour'][j]
            next_node = ant['tour'][j+1]
            # Pheromone deposition is symmetric for undirected graph
            tau[current_node, next_node] += delta_tau
            tau[next_node, current_node] += delta_tau
    return tau

# --- Plotting Functions ---

def _draw_edges(graph, ax, line_style='-k', line_width=1):
    """Helper function to draw all finite edges of the graph."""
    for i in range(graph['n_nodes']):
        for j in range(i + 1, graph['n_nodes']): # Avoid self-loops and duplicate edges
            if np.isfinite(graph['edges'][i, j]):
                x1, y1 = graph['nodes'][i]['x'], graph['nodes'][i]['y']
                x2, y2 = graph['nodes'][j]['x'], graph['nodes'][j]['y']
                ax.plot([x1, x2], [y1, y2], line_style, linewidth=line_width)

def _draw_nodes(graph, ax):
    """Helper function to draw all nodes of the graph."""
    for i in range(graph['n_nodes']):
        x, y = graph['nodes'][i]['x'], graph['nodes'][i]['y']
        # 'ok' marker, blue edge, cyan face, specific size and line width
        ax.plot(x, y, 'ok', markersize=10, markeredgecolor='b', linewidth=1, markerfacecolor='c')

def _draw_deleted_edge(graph, eliminated_nodes, ax):
    """Helper function to draw the eliminated edge in light gray."""
    if eliminated_nodes[0] != 0 and eliminated_nodes[1] != 0: # Check if nodes are valid (not the initial [0,0] placeholder)
        node1_idx, node2_idx = eliminated_nodes[0], eliminated_nodes[1]
        x1, y1 = graph['nodes'][node1_idx]['x'], graph['nodes'][node1_idx]['y']
        x2, y2 = graph['nodes'][node2_idx]['x'], graph['nodes'][node2_idx]['y']
        # Draw the deleted edge in light gray with thinner line
        ax.plot([x1, x2], [y1, y2], '-', color=(0.5, 0.5, 0.5), linewidth=0.5)

def draw_graph(graph, edge_eliminated, eliminated_nodes, ax):
    """
    Visualizes the nodes and edges of the graph.

    Parameters:
    graph (dict): The graph structure.
    edge_eliminated (bool): True if an edge has been eliminated.
    eliminated_nodes (list): A list [node1, node2] representing the eliminated edge (0-based indices).
    ax (matplotlib.axes.Axes): The axes object to draw on.
    """
    ax.clear() # Clear the current axes before redrawing
    _draw_edges(graph, ax) # Draw all current active edges
    if edge_eliminated:
        _draw_deleted_edge(graph, eliminated_nodes, ax) # Draw the eliminated edge
    _draw_nodes(graph, ax) # Draw all nodes

    ax.set_title('All Nodes and Edges')
    ax.set_xticks([]) # Remove x-axis ticks
    ax.set_yticks([]) # Remove y-axis ticks
    ax.set_aspect('equal', adjustable='box') # Maintain aspect ratio
    # Ensure the box is drawn by setting spine visibility
    for spine in ax.spines.values():
        spine.set_visible(True)

def draw_best_tour(queen_tour, graph, edge_eliminated, eliminated_nodes, ax):
    """
    Visualizes the nodes, edges, and the best tour (queen's tour) found so far.

    Parameters:
    queen_tour (list): The list of node indices (0-based) for the best tour.
    graph (dict): The graph structure.
    edge_eliminated (bool): True if an edge has been eliminated.
    eliminated_nodes (list): A list [node1, node2] representing the eliminated edge (0-based indices).
    ax (matplotlib.axes.Axes): The axes object to draw on.
    """
    ax.clear() # Clear the current axes before redrawing
    _draw_edges(graph, ax) # Draw all current active edges

    # Draw the best tour in red with thicker lines
    if len(queen_tour) > 1:
        for i in range(len(queen_tour) - 1):
            current_node = queen_tour[i]
            next_node = queen_tour[i+1]
            x1, y1 = graph['nodes'][current_node]['x'], graph['nodes'][current_node]['y']
            x2, y2 = graph['nodes'][next_node]['x'], graph['nodes'][next_node]['y']
            ax.plot([x1, x2], [y1, y2], '-r', linewidth=2)

    if edge_eliminated:
        _draw_deleted_edge(graph, eliminated_nodes, ax) # Draw the eliminated edge

    _draw_nodes(graph, ax) # Draw all nodes

    ax.set_title('Best Tour (The Queen)')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal', adjustable='box')
    # Ensure the box is drawn by setting spine visibility
    for spine in ax.spines.values():
        spine.set_visible(True)

def draw_pheromone(tau, graph, edge_eliminated, eliminated_nodes, ax):
    """
    Visualizes the nodes, edges, and the pheromone concentration on each edge.
    Pheromone concentration is mapped to line width and color intensity.

    Parameters:
    tau (np.ndarray): The pheromone matrix.
    graph (dict): The graph structure.
    edge_eliminated (bool): True if an edge has been eliminated.
    eliminated_nodes (list): A list [node1, node2] representing the eliminated edge (0-based indices).
    ax (matplotlib.axes.Axes): The axes object to draw on.
    """
    ax.clear() # Clear the current axes before redrawing
    _draw_edges(graph, ax) # Draw all current active edges

    # Normalize pheromone levels for visualization
    max_tau = np.max(tau)
    min_tau = np.min(tau)
    # Avoid division by zero if all tau values are the same
    if (max_tau - min_tau) == 0:
        tau_normalized = np.zeros_like(tau)
    else:
        tau_normalized = (tau - min_tau) / (max_tau - min_tau)

    # Draw edges with varying color and thickness based on pheromone
    for i in range(graph['n_nodes']):
        for j in range(i + 1, graph['n_nodes']): # Avoid self-loops and duplicate edges
            if np.isfinite(graph['edges'][i, j]):
                x1, y1 = graph['nodes'][i]['x'], graph['nodes'][i]['y']
                x2, y2 = graph['nodes'][j]['x'], graph['nodes'][j]['y']
                
                # Pheromone visualization: color and line width
                norm_tau = tau_normalized[i, j]
                # Matlab color format: [0, 0, (1-norm_tau), norm_tau] interpreted as RGBA
                # Here, we use a blue gradient, where higher pheromone means more opaque blue
                color = (0, 0, 1 - norm_tau, norm_tau)
                # Line width scales with normalized pheromone, with a minimum thickness
                line_width = 10.0 * norm_tau + 1
                
                ax.plot([x1, x2], [y1, y2], color=color, linewidth=line_width)

    if edge_eliminated:
        _draw_deleted_edge(graph, eliminated_nodes, ax) # Draw the eliminated edge

    _draw_nodes(graph, ax) # Draw all nodes

    ax.set_title('All Pheromones')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal', adjustable='box')
    # Ensure the box is drawn by setting spine visibility
    for spine in ax.spines.values():
        spine.set_visible(True)


# --- Main ACO Loop ---

def aco_loop(graph, max_iter, ant_no, tau, eta, alpha, beta, rho, edge_eliminated, deleted_nodes, fig, axes):
    """
    Main loop for the Ant Colony Optimization algorithm.
    Iteratively creates ant colonies, evaluates tours, updates pheromone levels,
    and visualizes the progress.

    Parameters:
    graph (dict): The graph structure.
    max_iter (int): Maximum number of iterations.
    ant_no (int): Number of ants in the colony.
    tau (np.ndarray): Initial pheromone matrix.
    eta (np.ndarray): Heuristic information matrix.
    alpha (float): Pheromone influence exponent.
    beta (float): Heuristic influence exponent.
    rho (float): Pheromone evaporation rate.
    edge_eliminated (bool): Flag indicating if an edge has been eliminated.
    deleted_nodes (list): Nodes of the eliminated edge.
    fig (matplotlib.figure.Figure): The figure object for dynamic plotting.
    axes (np.ndarray): Array of axes objects for subplots.

    Returns:
    tuple: Updated pheromone matrix (tau) and the best tour found (best_tour).
    """
    best_fitness = np.inf
    best_tour = []

    # Enable interactive plotting for dynamic updates in the loop
    plt.ion() # Turn on interactive mode
    fig.show() # Display the figure

    for t in range(1, max_iter + 1):
        # 1. Create Ants: Each ant constructs a tour
        colony = create_colony(graph, ant_no, tau, eta, alpha, beta)
        
        # 2. Calculate fitness values for all ants' tours
        for ant in colony['ants']:
            # The ant's tour might be empty if it got stuck during creation
            if not ant['tour']: 
                ant['fitness'] = np.inf # Mark as invalid tour
                continue
            ant['fitness'] = fitness_function(ant['tour'], graph)

        # 3. Find the best ant (queen) in the current iteration
        # Filter out infinite fitness values to find the true minimum
        valid_fitnesses = [ant['fitness'] for ant in colony['ants'] if np.isfinite(ant['fitness'])]
        if valid_fitnesses:
            min_val = np.min(valid_fitnesses)
            min_index = next((i for i, ant in enumerate(colony['ants']) if ant['fitness'] == min_val), None)
            
            if min_val < best_fitness:
                best_fitness = min_val
                best_tour = colony['ants'][min_index]['tour']
        else:
            min_val = np.inf 


        # Update the colony's 'queen' with the overall best tour found so far
        colony['queen']['tour'] = best_tour
        colony['queen']['fitness'] = best_fitness
            
        # 4. Update pheromone matrix based on ants' tours
        tau = update_pheromone(tau, colony)

        # 5. Evaporation: Reduce pheromone levels on all edges
        # This simulates pheromone decaying over time, encouraging exploration
        tau = (1 - rho) * tau
        
        # Ensure pheromone levels don't drop too low (can prevent getting stuck)
        # and don't become excessively high (can lead to premature convergence)
        tau[np.isinf(tau)] = 1e-10 # Prevent inf/NaN if some paths never used or weights changed
        # Add a lower bound to pheromone to prevent paths from being completely forgotten
        tau[tau < 1e-6] = 1e-6


        # 6. Display and Visualize results - update every iteration
        outmsg = f"Iteration #{t}, Shortest length = {best_fitness:.2f}"
        print(outmsg)

        # Update the plots every iteration
        draw_graph(graph, edge_eliminated, deleted_nodes, axes[0])
        axes[0].set_title(f'Iteration #{t}\nAll Nodes and Edges') # Update title for first subplot

        draw_best_tour(colony['queen']['tour'], graph, edge_eliminated, deleted_nodes, axes[1])
        draw_pheromone(tau, graph, edge_eliminated, deleted_nodes, axes[2])
        
        # Re-draw the canvas and pause briefly to show updates
        fig.canvas.draw()
        fig.canvas.flush_events()
        plt.pause(0.01) # Short pause to allow plot to render

    # Turn off interactive mode at the end
    plt.ioff()
    plt.close(fig) # Close the figure to prevent it from showing twice at the very end
    # Re-display the final state using plt.show()
    fig_final, axes_final = plt.subplots(1, 3, figsize=(18, 6))
    draw_graph(graph, edge_eliminated, deleted_nodes, axes_final[0])
    draw_best_tour(colony['queen']['tour'], graph, edge_eliminated, deleted_nodes, axes_final[1])
    draw_pheromone(tau, graph, edge_eliminated, deleted_nodes, axes_final[2])
    plt.suptitle(f"Final State after {max_iter} Iterations", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
    plt.show()

    return tau, best_tour

## 2. Problem Preparation

Here, we set up our graph, defining the nodes and the connections (edges) between them, along with their associated weights.

In [ ]:
# --- Problem preparation ---

# Initialize flags for edge elimination
edge_eliminated = False
deleted_nodes = [0, 0] # Placeholder: Will store 0-based indices if an edge is deleted

# Node coordinates for the graph (21 nodes)
# Note: These are 0-indexed in Python internally
x_coords = [7, 5, 7, 9, 4, 6, 8, 10, 3, 5, 7, 9, 11, 4, 6, 8, 10, 5, 7, 9, 7]
y_coords = [1, 3, 3, 3, 5, 5, 5, 5, 7, 7, 7, 7, 7, 9, 9, 9, 9, 11, 11, 11, 13]

# Create the graph structure
graph = create_graph(x_coords, y_coords)

# Calculate the midpoints of the graph's predefined edges.
# This is used later to identify which edge the user "clicks" on.
x_midpoint_edge, y_midpoint_edge = get_midpoint_coordinates(x_coords, y_coords)

# --- Initial Plotting ---
# Create a figure and a set of subplots for visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.suptitle("Ant Colony Optimization for TSP", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap

# Draw the initial graph
draw_graph(graph, edge_eliminated, deleted_nodes, axes[0])

# --- ACO Algorithm Parameters ---

max_iter = 500 # Maximum number of iterations for the ACO algorithm
ant_no = 50    # Number of ants in the colony

# Initial pheromone concentration (tau0)
# Calculated based on the number of nodes and the average finite edge weight
finite_edge_weights = graph['edges'][np.isfinite(graph['edges'])]
if finite_edge_weights.size > 0:
    tau0 = 10.0 / (graph['n_nodes'] * np.mean(finite_edge_weights))
else:
    tau0 = 1.0 # Fallback if no finite edges

# Pheromone matrix (tau) - initialized with tau0 on existing edges, 0 elsewhere
tau = np.zeros((graph['n_nodes'], graph['n_nodes']))
finite_edges_indices = np.isfinite(graph['edges'])
tau[finite_edges_indices] = tau0

# Heuristic information matrix (eta) - inverse of edge weights (desirability)
# Higher desirability for shorter edges
eta = np.zeros_like(graph['edges'])
# Avoid division by zero for infinite weights, set desirability to 0 for non-existent edges
eta[finite_edges_indices] = 1.0 / graph['edges'][finite_edges_indices]

rho = 0.8   # Evaporation rate: proportion of pheromone that evaporates in each iteration
alpha = 0.5 # Pheromone influence parameter: how much pheromone affects path choice
beta = 0.5  # Heuristic influence parameter: how much heuristic info affects path choice

print("Starting initial ACO run...")

# --- Main loop of ACO (First Run) ---
# This call runs the ACO algorithm for the specified number of iterations
# and updates the pheromone matrix and finds the best tour.
tau, queen_tour = aco_loop(graph, max_iter, ant_no, tau, eta, alpha, beta, rho, edge_eliminated, deleted_nodes, fig, axes)

print("\nInitial ACO run completed.")

## 3. Second Part: Edge Deletion and Re-optimization

This section demonstrates a powerful feature of ACO: its adaptability to dynamic changes in the graph. We will simulate the deletion of an edge and then re-run the ACO algorithm. The algorithm can leverage the previously learned pheromone information, potentially leading to faster convergence to a new optimal solution.

**Note on User Interaction:**
The original Matlab code used `ginput` for user interaction to select an edge for deletion. In a non-interactive Colab environment, `ginput` cannot be directly used in the same way. For this demonstration, I've hardcoded the deletion of a specific edge (nodes 0 and 1, which corresponds to the first edge defined in `create_graph`) to show the re-optimization process. In a dedicated desktop application, `ginput` or similar event listeners would enable interactive edge selection.

In [ ]:
# Display instruction for the second part
print("\n--- SECOND PART: Edge Deletion and Re-optimization ---")
print("**Please click on the graph below to select an edge to delete.**")
print("The simulation will pause until you click.")

# Ensure the figure for the first run is still active or re-create it to show interaction
# The previous aco_loop closes the fig, so we need a new one for ginput.
fig_interactive, axes_interactive = plt.subplots(1, 3, figsize=(18, 6))
plt.suptitle("Interactive Edge Deletion", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# Draw the current state of the graph, best tour, and pheromone to guide the user's click
draw_graph(graph, edge_eliminated, deleted_nodes, axes_interactive[0])
draw_best_tour(queen_tour, graph, edge_eliminated, deleted_nodes, axes_interactive[1])
draw_pheromone(tau, graph, edge_eliminated, deleted_nodes, axes_interactive[2])
plt.ion() # Turn on interactive mode for ginput
plt.show() # Display the figure

# --- Added: Force redraw before ginput to ensure readiness ---
fig_interactive.canvas.draw()
fig_interactive.canvas.flush_events()
# -----------------------------------------------------------

try:
    # Get user click coordinates
    # `timeout=-1` makes it wait indefinitely until a click
    # `n=1` means it expects a single click
    clicked_points = plt.ginput(n=1, timeout=-1, show_clicks=True)
    if clicked_points:
        a, b = clicked_points[0] # Extract x, y from the first click
        print(f"Clicked at: x={a:.2f}, y={b:.2f}")

        # Check which edge is closest to the clicked point
        deleted_edge_index_interactive = find_closest_edge(a, b, x_midpoint_edge, y_midpoint_edge)
        deleted_nodes = get_nodes_from_edge_index(deleted_edge_index_interactive)

        # Delete the identified edge by setting its weight to infinity
        graph['edges'][deleted_nodes[0], deleted_nodes[1]] = np.inf
        graph['edges'][deleted_nodes[1], deleted_nodes[0]] = np.inf

        edge_eliminated = True # Set flag to true for plotting
        print(f"Edge between nodes {deleted_nodes[0]} and {deleted_nodes[1]} has been eliminated.")

    else:
        print("No click received. Proceeding without edge deletion.")
except Exception as e:
    print(f"An error occurred during interactive input: {e}")
    print("Proceeding without interactive edge deletion.")
    # Reset edge_eliminated flag if an error occurred or no click
    edge_eliminated = False 
finally:
    plt.ioff() # Turn off interactive mode
    plt.close(fig_interactive) # Close the interactive figure

# Re-draw the graph state with the (potentially) deleted edge before new ACO run
fig_part2, axes_part2 = plt.subplots(1, 3, figsize=(18, 6))
plt.suptitle("ACO After Edge Deletion and Re-optimization", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

draw_graph(graph, edge_eliminated, deleted_nodes, axes_part2[0])
draw_best_tour(queen_tour, graph, edge_eliminated, deleted_nodes, axes_part2[1]) # Display previous best tour with new graph
draw_pheromone(tau, graph, edge_eliminated, deleted_nodes, axes_part2[2]) # Display previous pheromone with new graph
plt.show() # Display the updated graph state before new ACO run


# --- New Parameters for ACO (re-optimization) ---
# The Matlab code adds tau0 to the existing pheromone matrix.
# This boosts the overall pheromone levels to re-energize exploration after a change.
# Note: finite_edges_indices refers to the original graph structure, 
# use finite_edges_indices_new for the updated graph.

# Re-calculate tau0 based on the initial state's average edge weight
initial_finite_edge_weights = create_graph(x_coords, y_coords)['edges'][np.isfinite(create_graph(x_coords, y_coords)['edges'])]
if initial_finite_edge_weights.size > 0:
    tau0_recalc = 10.0 / (graph['n_nodes'] * np.mean(initial_finite_edge_weights))
else:
    tau0_recalc = 1.0

finite_edges_indices_current = np.isfinite(graph['edges'])
tau[finite_edges_indices_current] = tau[finite_edges_indices_current] + tau0_recalc

# Re-calculate eta, as edge weights might have changed (due to deletion)
eta = np.zeros_like(graph['edges'])
finite_edges_indices_new = np.isfinite(graph['edges']) # Re-evaluate finite edges
eta[finite_edges_indices_new] = 1.0 / graph['edges'][finite_edges_indices_new]

max_iter_part2 = 250 # Reduced iterations for the second run, typical for re-optimization

print("\nStarting re-optimization ACO run...")

# --- New Loop of ACO (Re-optimization) ---
# Run the ACO algorithm again with the modified graph and updated parameters
tau, queen_tour = aco_loop(graph, max_iter_part2, ant_no, tau, eta, alpha, beta, rho, edge_eliminated, deleted_nodes, fig_part2, axes_part2)

print("\nRe-optimization ACO run completed.")

## 4. Conclusion

This notebook demonstrates the Ant Colony Optimization algorithm's application to the Traveling Salesman Problem. You've seen how the algorithm iteratively improves its solution by learning from the paths taken by virtual "ants" through pheromone deposition and evaporation.

The second part highlights ACO's robustness: when an edge is removed, the algorithm can efficiently adapt and find a new optimal path, demonstrating its utility in dynamic environments. This is in contrast to some other optimization algorithms that might require a full restart from scratch.

This implementation provides a clear, commented, and runnable example of ACO using only NumPy, making it suitable for educational purposes in a university course on computational intelligence or optimization.